# Fn to FC
> python module to convert a given Fn into FC automatically

In [ ]:
#| default_exp fn_to_fc

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import openai
import json
import inspect
from typing import Optional

In [ ]:
import glob
from llmcam.ytlive import capture_youtube_live_frame
from llmcam.gpt4v import ask_gpt4v

In [ ]:
def capture_youtube_live_frame_and_save() -> str:
    "Capture a jpeg file from YouTube Live and save in data directory"
    return str(capture_youtube_live_frame())

In [ ]:
def ask_gpt4v_about_image_file(path:str) -> str:
    "Tell all about quantitative information from a given image file"
    info = ask_gpt4v(path)
    return json.dumps(info)

In [ ]:
#| export
def tool_schema(func):
    """ automatically generate a schema from its parameters and docstring"""
    # Extract function name, docstring, and parameters
    func_name = func.__name__
    func_description = func.__doc__ or "No description provided."
    signature = inspect.signature(func)
    
    # Create parameters schema
    parameters_schema = {
        "type": "object",
        "properties": {},
        "required": []
    }
    
    # Populate properties and required fields
    for param_name, param in signature.parameters.items():
        param_type = param.annotation if param.annotation != inspect._empty else str
        # Map Python types to JSON Schema types
        json_type = "string" if param_type == str else "number" if param_type in [int, float] else "boolean"
        
        # Add parameter to schema
        parameters_schema["properties"][param_name] = {
            "type": json_type,
            "description": f"{param_name} parameter of type {param_type.__name__}"
        }
        
        # Mark as required if no default
        if param.default == inspect.Parameter.empty:
            parameters_schema["required"].append(param_name)
    
    # Build final tool schema
    tool_schema = [
        {
            "type": "function",
            "function": {
                "name": func_name,
                "description": func_description,
                "parameters": parameters_schema,
            }
        }
    ]
    return tool_schema

In [ ]:
# Environmental setting up
tools = [tool_schema(fn) for fn in (capture_youtube_live_frame_and_save, ask_gpt4v_about_image_file)]
messages = [{"role":"system", "content":"You are a helpful system administrator. Use the supplied tools to assist the user."}]

In [ ]:
#| export
# Support functions to handle tool response,where res == response.choices[0].message
def fn_name(res): return res.tool_calls[0].function.name
def fn_args(res): return json.loads(res.tool_calls[0].function.arguments)    
def fn_exec(res): return globals().get(fn_name(res))(**fn_args(res))
def fn_result_content(res):
    """Create a content containing the result of the function call"""
    content = dict()
    content.update(fn_args(res))
    content.update({fn_name(res): fn_exec(res)})
    return json.dumps(content)

In [ ]:
#| export
def complete(role, content, tool_call_id=None):
    "Send completion request with messages, and save the response in messages again"
    messages.append({"role":role, "content":content, "tool_call_id":tool_call_id})
    response = openai.chat.completions.create(model="gpt-4o", messages=messages, tools=tools)
    res = response.choices[0].message
    messages.append(res.to_dict())
    if res.to_dict().get('tool_calls'):
        complete(role="tool", content=fn_result_content(res), tool_call_id=res.tool_calls[0].id)
    return messages[-1]['role'], messages[-1]['content']

In [ ]:
#| eval: false
complete("user", "Hi, can you capture YouTube Live?")

BadRequestError: Error code: 400 - {'error': {'message': "Invalid type for 'tools[0]': expected an object, but got an array instead.", 'type': 'invalid_request_error', 'param': 'tools[0]', 'code': 'invalid_type'}}

In [ ]:
#| eval: false
complete("user", "Yes, please!")

In [ ]:
#| eval: false
complete("user", "Where is it located?")

In [ ]:
#| eval: false
complete("user", "can you read that file?") # FIXME

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()